## 固有表現認識

### 固有表現認識とは？

In [1]:
# !pip install spacy-alignments seqeval

#### データセットのダウンロード

In [2]:
from datasets import load_dataset

# データセットを読み込む
dataset = load_dataset("llm-book/ner-wikipedia-dataset")

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using custom data configuration default
Reusing dataset ner-wikipedia-dataset (/root/.cache/huggingface/datasets/llm-book___ner-wikipedia-dataset/default/0.0.0/184bcf9be66116e777f2f534436226d47348676c93ba20cca58933f1b2b3b782)
100%|██████████| 3/3 [00:00<00:00, 273.22it/s]


In [3]:
# データセットの形式と事例数を確認する
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['curid', 'text', 'entities'],
        num_rows: 4274
    })
    validation: Dataset({
        features: ['curid', 'text', 'entities'],
        num_rows: 534
    })
    test: Dataset({
        features: ['curid', 'text', 'entities'],
        num_rows: 535
    })
})


In [4]:
from pprint import pprint

# 訓練セットの最初の二つの事例を表示する
pprint(list(dataset["train"])[:2])

[{'curid': '3638038',
  'entities': [{'name': 'さくら学院', 'span': [0, 5], 'type': 'その他の組織名'},
               {'name': 'Ciao Smiles', 'span': [6, 17], 'type': 'その他の組織名'}],
  'text': 'さくら学院、Ciao Smilesのメンバー。'},
 {'curid': '1729527',
  'entities': [{'name': 'レクレアティーボ・ウェルバ', 'span': [17, 30], 'type': 'その他の組織名'},
               {'name': 'プリメーラ・ディビシオン', 'span': [32, 44], 'type': 'その他の組織名'}],
  'text': '2008年10月5日、アウェーでのレクレアティーボ・ウェルバ戦でプリメーラ・ディビシオンでの初得点を決めた。'}]


#### データセットの分析

In [5]:
from collections import Counter
import pandas as pd
from datasets import Dataset

def count_label_occurrences(dataset: Dataset) -> dict[str, int]:
    """固有表現タイプの出現回数をカウント"""
    # 各事例から固有表現タイプを抽出したlistを作成する
    entities = [
        e["type"] for data in dataset for e in data["entities"]
    ]
    
    # ラベルの表現回数が多い順に並べる
    label_counts = dict(Counter(entities).most_common())
    return label_counts
    
label_counts_dict = {}
for split in dataset: # 各分割セットを処理する
    label_counts_dict[split] = count_label_occurrences(dataset[split])
# DataFrame形式で表示する
df = pd.DataFrame(label_counts_dict)
df.loc["合計"] = df.sum()
display(df)

,train,validation,test
人名,2394,299,287
法人名,2006,231,248
地名,1769,184,204
政治的組織名,953,121,106
製品名,934,123,158
施設名,868,103,137
その他の組織名,852,99,100
イベント名,831,85,93
合計,10607,1245,1333


#### スパンの重なる固有表現の存在を判定

In [6]:
def has_overlap(spans: list[tuple[int, int]]) -> int:
    """スパンの重なる固有表現の存在を判定"""
    sorted_spans = sorted(spans, key=lambda x: x[0])
    for i in range(1, len(sorted_spans)):
        # 前のスパンの終了位置が現在のスパンの開始位置より大きい場合、
        # 重なっているとする
        if sorted_spans[i - 1][1] > sorted_spans[i][0]:
            return 1
    return 0
    
# 各分割セットでスパンの重なる固有表現がある事例数を数える
overlap_count = 0
for split in dataset: # 各分割セットを処理する
    for data in dataset[split]: # 各事例を処理する
        if data["entities"]: # 固有表現の存在しない事例はスキップ
            # スパンのみのlistを作成する
            spans = [e["span"] for e in data["entities"]]
            overlap_count += has_overlap(spans)
    print(f"{split}におけるスパンが重複する事例数:{overlap_count}")

trainにおけるスパンが重複する事例数:0
validationにおけるスパンが重複する事例数:0
testにおけるスパンが重複する事例数:0


#### 前処理

#### テキストの正規化

In [7]:
from unicodedata import normalize

# テキストに対してUnicode正規化を行う
text =  "ＡＢＣABCａｂｃabcｱｲｳアイウ①②③123"
normalized_text = normalize("NFKC", text)
print(f"正規化前: {text}")
print(f"正規化後: {normalized_text}")

正規化前: ＡＢＣABCａｂｃabcｱｲｳアイウ①②③123
正規化後: ABCABCabcabcアイウアイウ123123


In [8]:
# 文字列の長さが変わる場合ある
text = "㈱、3㌕、10℃"
normalized_text = normalize("NFKC", text)
print(f"正規化前: {text}")
print(f"正規化後: {normalized_text}")

正規化前: ㈱、3㌕、10℃
正規化後: (株)、3キログラム、10°C


In [9]:
from unicodedata import normalize, is_normalized

count = 0
for split in dataset: # 各分割セットを処理する
    for data in dataset[split]: # 各事例を処理する
        # テキストが正規化されていない事例をカウントする
        if not is_normalized("NFKC", data["text"]):
            count += 1
print(f"正規化されていない事例数: {count}")

正規化されていない事例数: 0


#### テキストのトークナイゼーション

In [10]:
from transformers import AutoTokenizer

# トークナイザを読み込み
model_name = "tohoku-nlp/bert-base-japanese-v3"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# トークナイゼーションを行う
subwords = "/".join(tokenizer.tokenize(dataset["train"][0]["text"]))
characters = "/".join(dataset["train"][0]["text"])
print(f"サブワード単位: {subwords}")
print(f"文単位: {characters}")

サブワード単位: さくら/学院/、/C/##ia/##o/Sm/##ile/##s/の/メンバー/。
文単位: さ/く/ら/学/院/、/C/i/a/o/ /S/m/i/l/e/s/の/メ/ン/バ/ー/。


#### 文字列とトークン列のアライメント

In [11]:
text = "さくら学院"

In [12]:
from spacy_alignments.tokenizations import get_alignments

# 文字列のlistを獲得する
characters = list(text)
# テキストを特殊トークンを含めたトークンのlistに変換する
tokens = tokenizer.convert_ids_to_tokens(tokenizer.encode(text))
# 文字のlistとトークンのlistのアライメントをとる
char_to_token_indices, token_to_char_indices = get_alignments(characters, tokens)
print(f"文字のlist: {characters}")
print(f"トークンのlist: {tokens}")
print(f"文字に対するトークンの位置: {char_to_token_indices}")
print(f"トークンに対する文字の位置: {token_to_char_indices}")

文字のlist: ['さ', 'く', 'ら', '学', '院']
トークンのlist: ['[CLS]', 'さくら', '学院', '[SEP]']
文字に対するトークンの位置: [[1], [1], [1], [2], [2]]
トークンに対する文字の位置: [[], [0, 1, 2], [3, 4], []]


#### 系列ラベリングのためのラベル作成

In [13]:
text = "大谷翔平は岩手県水沢市出身"
entities = [
    {"name": "大谷翔平", "span": [0,4], "type": "人名"},
    {"name": "岩手県水沢市", "span": [5,11], "type": "地名"},
]

In [14]:
from transformers import PreTrainedTokenizer

def output_tokens_and_labels(
    text: str,
    entities: list[dict[str, list[int] | str]],
    tokenizer: PreTrainedTokenizer,
) -> tuple[list[str], list[str]]:
    """トークンのlistとラベルのlistを出力"""
    # 文字列のlistとトークンのlistのアライメントをとる
    characters = list(text)
    tokens = tokenizer.convert_ids_to_tokens(tokenizer.encode(text))
    char_to_token_indices, _ = get_alignments(characters, tokens)
    
    # "O"のラベルで初期化したラベルのlistを作成する
    labels = ["O"] * len(tokens)
    for entity in entities: # 各固有表現で処理する
        entity_span, entity_type = entity["span"], entity["type"]
        start = char_to_token_indices[entity_span[0]][0]
        end = char_to_token_indices[entity_span[1]-1][0]
        # 固有表現の開始トークンの位置に"B-"のラベルを設定する
        labels[start] = f"B-{entity_type}"
        # 固有表現の開始トークン以外の位置に"I-"のラベルを設定する
        for idx in range(start + 1, end + 1):
            labels[idx] = f"I-{entity_type}"
    # 特殊トークンの位置にはラベルを設定しない
    labels[0] = "-" # 開始
    labels[-1] = "-" # 終了
    return tokens, labels

# トークンとラベルのlistを出力する
tokens, labels = output_tokens_and_labels(text, entities, tokenizer)
# DataFrameの形式で表示する
df = pd.DataFrame({"トークン列": tokens, "ラベル列": labels})
df.index.name = "位置"
display(df.T)

位置,0,1,2,3,4,5,6,7,8,9,10
トークン列,[CLS],大谷,翔,##平,は,岩手,県,水沢,市,出身,[SEP]
ラベル列,-,B-人名,I-人名,I-人名,O,B-地名,I-地名,I-地名,I-地名,O,-


#### 評価指標

#### seqevalライブラリを用いた評価スコアの算出

In [15]:
from typing import Any
from seqeval.metrics import classification_report

def create_character_labels(
    text: str, entities: list[dict[str, list[int] | str]]
) -> list[str]:
    """文字ベースでラベルのlistを作成"""
    # "O"のラベルで初期化したラベルのlistを作成する
    labels = ["O"] * len(text)
    for entity in entities: # 各固有表現を処理する
        entity_span, entity_type = entity["span"], entity["type"]
        # 固有表現の開始文字の位置に"B-"のラベルを設定する
        labels[entity_span[0]] = f"B-{entity_type}"
        # 固有表現の開始文字以外の位置に"I-"ラベルを設定する
        for i in range(entity_span[0] + 1, entity_span[1]):
            labels[i] = f"I-{entity_type}"
    return labels
    
def convert_results_to_labels(
    results: list[dict[str, Any]]
) -> tuple[list[list[str]], list[list[str]]]:
    """正解データと予測データのラベルのlistを作成"""
    true_labels, pred_labels = [], []
    for result in results: # 各事例を処理する
        # 文字ベースでラベルのリストを作成してlistに加える
        true_labels.append(
            create_character_labels(result["text"], result["entities"])
        )
        pred_labels.append(
            create_character_labels(result["text"], result["pred_entities"])
        )
    return true_labels, pred_labels

In [16]:
results = [
    {
        "text": "大谷翔平は岩手県水沢市出身",
        "entities": [
            {"name": "大谷翔平", "span": [0,4], "type": "人名"},
            {"name": "岩手県水沢市", "span": [5, 11], "type": "地名"}
        ],
        "pred_entities": [
            {"name": "大谷翔平", "span": [0,4], "type": "人名"},
            {"name": "岩手県", "span": [5,8], "type": "地名"},
            {"name": "水沢市", "span": [8,11], "type": "施設名"}
        ],
    }
]

# 正解データと予測データのラベルのlistを作成
true_labels, pred_labels = convert_results_to_labels(results)
# 評価結果を取得して表示
print(classification_report(true_labels, pred_labels))

              precision    recall  f1-score   support

          人名       1.00      1.00      1.00         1
          地名       0.00      0.00      0.00         1
         施設名       0.00      0.00      0.00         0

   micro avg       0.33      0.50      0.40         2
   macro avg       0.33      0.33      0.33         2
weighted avg       0.50      0.50      0.50         2



/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
from seqeval.metrics import f1_score, precision_score, recall_score

def compute_scores(
    true_labels: list[list[str]], pred_labels: list[list[str]], average: str
) -> dict[str, float]:
    """適合率、再現率、F値を算出"""
    scores = {
        "precision": precision_score(true_labels, pred_labels, average=average),
        "recall": recall_score(true_labels, pred_labels, average=average),
        "f1-score": f1_score(true_labels, pred_labels, average=average),
    }
    return scores

# 適合率、再現率、F値のマイクロ平均を算出する
print(compute_scores(true_labels, pred_labels, "micro"))
# 適合率、再現率、F値のマクロ平均を算出する
print(compute_scores(true_labels, pred_labels, "macro"))

{'precision': 0.3333333333333333, 'recall': 0.5, 'f1-score': 0.4}
{'precision': 0.3333333333333333, 'recall': 0.3333333333333333, 'f1-score': 0.3333333333333333}


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### 固有表現認識モデルの実装

#### BERTのファインチューニング

In [18]:
# ラベルとIDを対応付けるdictの作成
import torch

def create_label2id(
    entities_list: list[list[dict[str, str | int]]]
) -> dict[str, int]:
    """ラベルとIDを紐づけるdictを作成"""
    # "O"のIDには0を割り当てる
    label2id = {"O": 0}
    # 固有表現タイプのsetを獲得して並び替える
    entity_types = set(
        [e["type"] for entities in entities_list for e in entities]
    )
    entity_types = sorted(entity_types)
    for i, entity_type in enumerate(entity_types):
        # "B-"のIDには奇数番号を割り当てる
        label2id[f"B-{entity_type}"] = i * 2 + 1
        # "I-"のIDには偶数番号を割り当てる
        label2id[f"I-{entity_type}"] = i * 2 + 2
    return label2id

# ラベルとIDを紐づけるdictを作成する
label2id = create_label2id(dataset["train"]["entities"])
id2label = {v:k for k, v in label2id.items()}
pprint(id2label)

{0: 'O',
 1: 'B-その他の組織名',
 2: 'I-その他の組織名',
 3: 'B-イベント名',
 4: 'I-イベント名',
 5: 'B-人名',
 6: 'I-人名',
 7: 'B-地名',
 8: 'I-地名',
 9: 'B-政治的組織名',
 10: 'I-政治的組織名',
 11: 'B-施設名',
 12: 'I-施設名',
 13: 'B-法人名',
 14: 'I-法人名',
 15: 'B-製品名',
 16: 'I-製品名'}


#### データの前処理

In [19]:
from transformers.tokenization_utils_base import BatchEncoding

def preprocess_data(
    data: dict[str, Any],
    tokenizer: PreTrainedTokenizer,
    label2id: dict[int, str],
) -> BatchEncoding:
    """データの前処理"""
    # テキストのトークナイゼーションを行う
    inputs = tokenizer(
        data["text"],
        return_tensors="pt",
        return_special_tokens_mask=True,
    )
    inputs = {k: v.squeeze(0) for k, v in inputs.items()}
    
    # 文字のlistとトークンのlistのアライメントをとる
    characters = list(data["text"])
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"])
    char_to_token_indices, _ = get_alignments(characters, tokens)
    
    # "O"のIDのlistを作成する
    labels = torch.zeros_like(inputs["input_ids"])
    for entity in data["entities"]: # 各固有表現を処理する
        start_token_indices = char_to_token_indices[entity["span"][0]]
        end_token_indices = char_to_token_indices[
            entity["span"][1] - 1
        ]
        # 文字に対するトークンが存在しなければスキップする
        if (
            len(start_token_indices)==0
            or len(end_token_indices)==0
        ):
            continue
        start, end = start_token_indices[0], end_token_indices[0]
        entity_type = entity["type"]
        # 固有表現の開始トークンの位置に"B-"のIDを設定する
        labels[start] = label2id[f"B-{entity_type}"]
        # 固有表現の開始トークン以外の位置に"I-"のIDを設定する
        if start != end:
            labels[start + 1: end + 1] = label2id[f"I-{entity_type}"]
    # 特殊トークンの位置のIDは-100とする
    labels[torch.where(inputs["special_tokens_mask"])] = -100
    inputs["labels"] = labels
    return inputs
    
# 訓練セットに対して前処理を行う
train_dataset = dataset["train"].map(
    preprocess_data,
    fn_kwargs={
        "tokenizer": tokenizer,
        "label2id": label2id,
    },
    remove_columns=dataset["train"].column_names,
)
# 検証セットに対して前処理を行う
validation_dataset = dataset["validation"].map(
    preprocess_data,
    fn_kwargs={
        "tokenizer": tokenizer,
        "label2id": label2id,
    },
    remove_columns=dataset["validation"].column_names,
)

100%|██████████| 534/534 [00:00<00:00, 1397.92ex/s]


#### モデルの準備

In [20]:
from transformers import (
    AutoModelForTokenClassification,
    DataCollatorForTokenClassification,
)

# モデルを読み込む
model = AutoModelForTokenClassification.from_pretrained(
    model_name, label2id=label2id, id2label=id2label
)
# パラメータをメモリ上に隣接する形で配置
for param in model.parameters():
    param.data = param.data.contiguous()
# collate関数にDataCollatorForTokenClassificationを用いる
data_collator = DataCollatorForTokenClassification(tokenizer)

/usr/local/lib/python3.10/dist-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https:/

#### モデルのファインチューニング

In [21]:
from transformers import Trainer, TrainingArguments
from transformers.trainer_utils import set_seed

# 乱数シードを42に固定する
set_seed(42)

# Trainerに渡す引数を初期化する
training_args = TrainingArguments(
    output_dir="../model/output_bert_ner",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=1e-4,
    lr_scheduler_type="linear",
    warmup_ratio=0.1,
    num_train_epochs=5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    fp16=True,
    report_to="none",
)

# Trainerを初期化する
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    data_collator=data_collator,
    args=training_args,
)

# 訓練する
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_1678/359011269.py:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.654500,0.098389
2,0.068500,0.091702
3,0.031200,0.091689
4,0.011600,0.100583
5,0.005700,0.104642


TrainOutput(global_step=670, training_loss=0.15430139712433316, metrics={'train_runtime': 102.5866, 'train_samples_per_second': 208.312, 'train_steps_per_second': 6.531, 'total_flos': 1070012411245680.0, 'train_loss': 0.15430139712433316, 'epoch': 5.0})

### 固有表現の予測・抽出

#### 固有表現ラベルの予測

In [34]:
def convert_list_dict_to_dict_list(
    list_dict: dict[str, list]
) -> list[dict[str, list]]:
    """ミニバッチのデータを事例単位のlistに変換"""
    dict_list = []
    # dictのキーのlistを作成する
    keys = list(list_dict.keys())
    for idx in range(len(list_dict[keys[0]])): # 各事例で処理する
        # dictの各キーからデータを取り出してlistに追加する
        dict_list.append({key: list_dict[key][idx] for key in keys})
    return dict_list

# ミニバッチのデータを事例単位のlistに変換する
list_dict = {
    "input_ids": [[0, 1], [2, 3]],
    "labels": [[1, 2], [3, 4]],
}
dict_list = convert_list_dict_to_dict_list(list_dict)
print(f"入力: {list_dict}")
print(f"出力: {dict_list}")

入力: {'input_ids': [[0, 1], [2, 3]], 'labels': [[1, 2], [3, 4]]}
出力: [{'input_ids': [0, 1], 'labels': [1, 2]}, {'input_ids': [2, 3], 'labels': [3, 4]}]


In [36]:
from tqdm import tqdm
from torch.utils.data import DataLoader
from transformers import PreTrainedModel

def run_prediction(
    dataloader: DataLoader, 
    model: PreTrainedModel
) -> list[dict[str, Any]]:
    """予測スコアに基づき固有表現ラベルを予測"""
    predictions = []
    for batch in tqdm(dataloader): # 各ミニバッチを処理する
        inputs = {
            k: v.to(model.device)
            for k, v in batch.items()
            if k != "special_tokens_mask"
        }
        # 予測スコアを取得する
        logits = model(**inputs).logits
        # 最もスコアの高いIDを取得する
        batch["pred_label_ids"] = logits.argmax(-1)
        batch = {k: v.cpu().tolist() for k, v in batch.items()}
        # ミニバッチのデータを事例単位のlistに変換する
        predictions += convert_list_dict_to_dict_list(batch)
    return predictions

# ミニバッチの作成にDataLoaderを用いる
validation_dataloader = DataLoader(
    validation_dataset,
    batch_size=32,
    shuffle=False,
    collate_fn=data_collator,
)
# 固有表現ラベルを予測する
predictions = run_prediction(validation_dataloader, model)
print(predictions[0]["pred_label_ids"])

100%|██████████| 17/17 [00:00<00:00, 39.61it/s]

[0, 0, 15, 16, 0, 0, 13, 14, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 13, 14, 14, 14, 13, 0, 0, 0, 0, 0, 0, 15, 16, 15, 13, 14, 14, 14, 14, 13, 13, 13, 14, 14, 0, 0, 0, 13, 13, 14, 14, 14, 0, 0, 13, 14, 14, 0, 0, 0, 0, 0, 0, 15, 16, 16, 0, 13, 14, 14, 14, 14, 15, 0, 0, 15, 15, 15, 16, 16, 0, 13, 14]


#### 固有表現の抽出

In [39]:
from seqeval.metrics.sequence_labeling import get_entities

def extract_entities(
    predictions: list[dict[str, Any]],
    dataset: list[dict[str, Any]],
    tokenizer: PreTrainedTokenizer,
    id2label: dict[int, str],
) -> list[dict[str, Any]]:
    """固有表現を抽出"""
    results = []
    for prediction, data in zip(predictions, dataset):
        # 文字列のlistを取得する
        characters = list(data["text"])
        
        # 特殊トークンを除いたトークンのlistと予測ラベルのlistを取得する
        tokens, pred_labels = [], []
        all_tokens = tokenizer.convert_ids_to_tokens(
            prediction["input_ids"]
        )
        for token, label_id in zip(
            all_tokens, prediction["pred_label_ids"]
        ):
            # 特殊トークン以外をlistに追加する
            if token not in tokenizer.all_special_tokens:
                tokens.append(token)
                pred_labels.append(id2label[label_id])
                
        # 文字のlistとトークンのlistのアライメントを取る
        _, token_to_char_indices = get_alignments(characters, tokens)
        
        # 予測ラベルのlistから固有表現タイプと、
        # トークン単位の開始位置と終了位置を取得して、
        # それらを正解データと同じ形式にする
        pred_entities = []
        for entity in get_entities(pred_labels):
            entity_type, token_start, token_end = entity
            # 文字単位の開始位置を取得する
            char_start = token_to_char_indices[token_start][0]
            # 文字単位の終了位置を取得する
            char_end = token_to_char_indices[token_end][-1] + 1
            pred_entity = {
                "name": "".join(characters[char_start:char_end]),
                "span": [char_start, char_end],
                "type": entity_type,
            }
            pred_entities.append(pred_entity)
        data["pred_entities"] = pred_entities
        results.append(data)
    return results

# 固有表現を抽出する
results = extract_entities(
    predictions, dataset["validation"], tokenizer, id2label
)
pprint(results[0])

{'curid': '1662110',
 'entities': [{'name': '復活篇', 'span': [1, 4], 'type': '製品名'},
              {'name': 'グリーンバニー', 'span': [6, 13], 'type': '法人名'}],
 'pred_entities': [{'name': '復活篇', 'span': [1, 4], 'type': '製品名'},
                   {'name': 'グリーンバニー', 'span': [6, 13], 'type': '法人名'}],
 'text': '「復活篇」はグリーンバニーからの発売となっている。'}
